In [52]:
# Sage Math Implementation of Feige Fiat Shamir Zero Knowledge Identification Scheme

# To get a different random number everytime
import time
seed = int(round(time.time() * 10))


# Generator for random number generation
def gen_random_mod(k, modulus):
    num = 0
    while(num < k):
        a = mod(gp.random(), modulus).lift()
        while(gcd(a, modulus) != 1):
            a = mod(gp.random(), modulus).lift()
        yield a
        num += 1

class ZKP_Setup_TTP:
    
    def __init__(self, modbits, k, seed):
        self.k = k
        set_random_seed(seed)
        current_randstate().set_seed_gp()
        # use gp.random() to get prng random number
        self.p = next_prime(2^modbits + gp.random())
        print("p : ", self.p)
        self.q = next_prime(2^modbits + gp.random())
        print("q : ", self.q)
        if self.p == self.q:
            print("Primes are same, please run again")
        self.n = self.p*self.q
        print("n = "+str(self.n))
        #a = [ i-1 for i in list(gen_random_mod(30, 3))]
        self.a = list(gen_random_mod(k, self.n))
        assert sum([gcd(i, self.n) for i in self.a]) == len(self.a)
        #print(a)
        self.asq = [i.powermod(2, self.n) for i in self.a]
        #print(asq)
        
class ZKP_Peggy:
    
    def __init__(self, n, sk):
        self.n = n
        self.sk = sk
        self.k = len(sk)
        #print(self.sk)
        
    def x_compute(self):
        sign = [ i-1 for i in list(gen_random_mod(1, 3))]
        rlist = list(gen_random_mod(1, self.n))
        self.r = rlist[0]
        rsq = self.r.powermod(2, self.n)
        if(sign[0] == 0):
            return rsq
        else:
            return self.n-rsq
        
    def y_compute(self, alist):
        y = mod(1, self.n)
        y = y*self.r
        for i in range(self.k):
            y = y*self.sk[i].powermod(alist[i], self.n)
        y = y.lift()
        assert y < self.n
        return y
        

class ZKP_Victor:
    
    def __init__(self, n, vk):
        self.n = n
        self.vk = vk
        self.k = len(vk)
        #print(self.vk)
        
    def choice_a(self, x):
        self.abort = 0
        if x == 0:
            self.abort = 1
            return []
        self.x = x
        self.a = [ i-1 for i in list(gen_random_mod(k, 3))]
        #print(self.a)
        return self.a
    
    def authenticate(self, y):
        ysq = y.powermod(2, self.n)
        print("y^2 value to be checked = " + str(ysq))
        yrhs = mod(1, self.n)
        yrhs = yrhs*self.x
        for i in range(self.k):
            yrhs = yrhs*self.vk[i].powermod(self.a[i], self.n)
        yrhs = yrhs.lift()
        if yrhs == ysq or yrhs == self.n-ysq:
            return 0
        else:
            return 1

class ZKP_Protocol:
    
    def __init__(self, modbits, k, seed, iterations):
        zkp_gen = ZKP_Setup_TTP(modbits, k, seed)
        peggy = ZKP_Peggy(zkp_gen.n, zkp_gen.a)
        victor = ZKP_Victor(zkp_gen.n, zkp_gen.asq)
        #Now protocol begins
        cnt = 0
        abort = 0
        while (cnt < iterations and abort == 0):
            print("-------------------Iteration number " + str(cnt+1) + "------------------")
            x = peggy.x_compute()
            alist = victor.choice_a(x)
            if not alist:
                print("x was 0, so cannot proceed! Authentication Fails.")
                abort = 1
            else:
                y = peggy.y_compute(alist)
                abort = victor.authenticate(y)
            print("One Iteration of the protocol ended.")
            if(abort == 1):
                print("Failed at iteration " + str(cnt+1))
            cnt += 1
        print("----------------------Ended------------------------")
        if abort == 0:
            print("Authentication Successful. Zero Knowledge Proof presented by Peggy was indeed correct.")
            print("No information about the secret sk was leaked to Victor in this process.")
        elif abort == 1:
            print("Authentication UNSUCCESSFUL! Zero Knowledge Proof presented by Peggy was indeed INCORRECT.")
        
modbits = 12
k = 10
iterations = 10
zkp = ZKP_Protocol(modbits, k, seed, iterations)

('p : ', 146715971)
('q : ', 1420438457)
n = 208401007464496747
-------------------Iteration number 1------------------
y^2 value to be checked = 140436635875085226
One Iteration of the protocol ended.
-------------------Iteration number 2------------------
y^2 value to be checked = 58389631378839117
One Iteration of the protocol ended.
-------------------Iteration number 3------------------
y^2 value to be checked = 100045907517066627
One Iteration of the protocol ended.
-------------------Iteration number 4------------------
y^2 value to be checked = 152716804525263854
One Iteration of the protocol ended.
-------------------Iteration number 5------------------
y^2 value to be checked = 51289082721150079
One Iteration of the protocol ended.
-------------------Iteration number 6------------------
y^2 value to be checked = 107369114521425733
One Iteration of the protocol ended.
-------------------Iteration number 7------------------
y^2 value to be checked = 190258694883471547
One Itera